In [18]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from scipy import stats
import xgboost as xgb


from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

input_path='C:\\Users\\varsh\\Documents\\Kaggle\\Purchase Prediction\\varshini_share\\input\\'
output_path='C:\\Users\\varsh\\Documents\\Kaggle\\Purchase Prediction\\varshini_share\\output\\'

## 1). Data Preparation

In [19]:
#Importing the data
train=pd.read_csv(input_path+"train.csv")
test=pd.read_csv(input_path+"test.csv")
print(train.shape)
print(test.shape)

#Combining train and test in to a single dataframe to do the data prep together
frames = [train, test]
my_data = pd.concat(frames,keys=['train', 'test'])
print(my_data.shape)

(665249, 25)
(198856, 25)
(864105, 25)


In [20]:
#Imputing the NAs from vairables
my_data['location'] = my_data['location'].fillna(0)
my_data['C_previous'] = my_data['C_previous'].fillna(99)
my_data['duration_previous'] = my_data['duration_previous'].fillna(99)
my_data['car_value'] = my_data['car_value'].fillna(99)

#Risk factor has a lot of NA values and remaining two are ID variables
cols_to_remove=['location','risk_factor']
my_data=my_data.drop(cols_to_remove,axis=1)

#printing column wise null value counts
print("Column-wise Null value counts")
my_data.isnull().sum(axis = 0)

Column-wise Null value counts


customer_ID          0
shopping_pt          0
record_type          0
day                  0
time                 0
state                0
group_size           0
homeowner            0
car_age              0
car_value            0
age_oldest           0
age_youngest         0
married_couple       0
C_previous           0
duration_previous    0
A                    0
B                    0
C                    0
D                    0
E                    0
F                    0
G                    0
cost                 0
dtype: int64

### One hot encoding of catergorical columns and converting time in to a cyclical variable

In [21]:
#Function for one hot encoding
def one_hot(df, cols):
    for col in cols:
        df[col] = df[col].astype('category')
    df=pd.concat([df,pd.get_dummies(df[cols])], axis=1)
    df.drop(cols, axis=1, inplace=True)
    return df

#Function to convert time in to a cyclical variable
def time_convert(df):
    df['time'] = df['time'].apply(lambda x: (int(x[:2]) * 60) + int(x[3:]))
    df['time_sin'] = np.sin((df['time'] * np.pi) / 1800)
    df['time_cos'] = np.cos((df['time'] * np.pi) / 1800)
    df = df.drop(['time'], axis=1)
    return df

In [22]:
#Applying the above functions
# cat_cols = ['shopping_pt','day', 'state',
#        'group_size', 'homeowner', 'car_age','car_value', 'age_oldest', 'age_youngest',
#        'married_couple','C_previous','duration_previous']

cat_cols = ['shopping_pt','day', 'state','group_size', 'homeowner','car_value','married_couple','C_previous']

my_data = one_hot(my_data,cat_cols)
my_data = time_convert(my_data)

### Splitting the dataset in to Test and Train

In [23]:
#Split data back in to train & Test
my_train=my_data.loc['train']
my_test=my_data.loc['test']
print(my_train.shape)
print(my_test.shape)

(665249, 95)
(198856, 95)


### Function that Lags the policy information to get the Previous 'K' policies as columns to our data - Both Train &Test data

In [24]:
# Experiment with different values of K
K=3

cols_to_lag=['A','B','C','D','E','F','G']
lagged_cols=[]

#Lagging
for col in cols_to_lag:
    for i in range(1,K):
        lagged_cols.append('{}_previous_{}'.format(col,i))
        my_train.loc[:,'{}_previous_{}'.format(col,i)]=my_train.loc[:,'{}'.format(col)].shift(i).fillna(99)
        my_test.loc[:,'{}_previous_{}'.format(col,i)]=my_test['{}'.format(col)].shift(i).fillna(99)
frames = [my_train, my_test]
my_data = pd.concat(frames,keys=['train', 'test'])
print(my_data.shape)


#one-hot encoding the lagged variables
my_data = one_hot(my_data,lagged_cols)
my_train=my_data.loc['train']
my_test=my_data.loc['test']
print(my_train.shape)
print(my_test.shape)

C:\Users\varsh\AppData\Local\Temp\ipykernel_40536\2692081912.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_train.loc[:,'{}_previous_{}'.format(col,i)]=my_train.loc[:,'{}'.format(col)].shift(i).fillna(99)
C:\Users\varsh\AppData\Local\Temp\ipykernel_40536\2692081912.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_test.loc[:,'{}_previous_{}'.format(col,i)]=my_test['{}'.format(col)].shift(i).fillna(99)
C:\Users\varsh\AppData\Local\Temp\ipykernel_40536\2692081912.py:11: SettingWithCopyWarning: 

(864105, 109)
(665249, 153)
(198856, 153)


In [25]:
my_train.head()

,customer_ID,record_type,car_age,age_oldest,age_youngest,duration_previous,A,B,C,D,...,G_previous_1_1.0,G_previous_1_2.0,G_previous_1_3.0,G_previous_1_4.0,G_previous_1_99.0,G_previous_2_1.0,G_previous_2_2.0,G_previous_2_3.0,G_previous_2_4.0,G_previous_2_99.0
0,10000000,0,2,46,42,2.0,1,0,2,2,...,0,0,0,0,1,0,0,0,0,1
1,10000000,0,2,46,42,2.0,1,0,2,2,...,0,1,0,0,0,0,0,0,0,1
2,10000000,0,2,46,42,2.0,1,0,2,2,...,1,0,0,0,0,0,1,0,0,0
3,10000000,0,2,46,42,2.0,1,0,2,2,...,1,0,0,0,0,1,0,0,0,0
4,10000000,0,2,46,42,2.0,1,0,2,2,...,1,0,0,0,0,1,0,0,0,0


In [26]:
my_test.head()

,customer_ID,record_type,car_age,age_oldest,age_youngest,duration_previous,A,B,C,D,...,G_previous_1_1.0,G_previous_1_2.0,G_previous_1_3.0,G_previous_1_4.0,G_previous_1_99.0,G_previous_2_1.0,G_previous_2_2.0,G_previous_2_3.0,G_previous_2_4.0,G_previous_2_99.0
0,10000001,0,9,24,24,9.0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
1,10000001,0,9,24,24,9.0,2,1,1,3,...,0,0,0,1,0,0,0,0,0,1
2,10000002,0,7,74,74,15.0,2,0,2,3,...,0,1,0,0,0,0,0,0,1,0
3,10000002,0,7,74,74,15.0,2,0,2,3,...,0,1,0,0,0,0,1,0,0,0
4,10000003,0,4,26,26,1.0,1,0,1,1,...,0,1,0,0,0,0,1,0,0,0


### TRAIN DATA-  Filtering only the rows with the purchased flag(record type=1) and renaming the policy variables to "Purchased" - which will the response variable for each indivdual model

In [27]:
my_train = my_train[my_train['record_type'] == 1]

#renaming the columns to "{policyname}_purchased"
my_train=my_train.rename(columns={"A": "A_purchased","B": "B_purchased","C": "C_purchased","D": "D_purchased",
                                  "E": "E_purchased","F": "F_purchased","G": "G_purchased"})



# options=['A','B','C','D','E','F','G']
# for col in options:
#      my_train['{}_purchased'.format(col)]=my_train[col]

my_train.head()

,customer_ID,record_type,car_age,age_oldest,age_youngest,duration_previous,A_purchased,B_purchased,C_purchased,D_purchased,...,G_previous_1_1.0,G_previous_1_2.0,G_previous_1_3.0,G_previous_1_4.0,G_previous_1_99.0,G_previous_2_1.0,G_previous_2_2.0,G_previous_2_3.0,G_previous_2_4.0,G_previous_2_99.0
8,10000000,1,2,46,42,2.0,1,0,2,2,...,1,0,0,0,0,1,0,0,0,0
14,10000005,1,10,28,28,13.0,0,0,3,2,...,0,1,0,0,0,0,1,0,0,0
22,10000007,1,11,43,43,4.0,0,0,1,2,...,1,0,0,0,0,1,0,0,0,0
26,10000013,1,3,62,60,3.0,1,1,3,2,...,0,0,1,0,0,0,0,1,0,0
32,10000014,1,5,32,28,2.0,1,1,1,1,...,0,1,0,0,0,0,1,0,0,0


### TEST DATA - Retaining only the purchased data rows for the model

In [28]:
#We are taking the last row for each customer from the test data
my_test=my_test.groupby('customer_ID').last()
my_test=my_test.reset_index()
my_test.head()

,customer_ID,record_type,car_age,age_oldest,age_youngest,duration_previous,A,B,C,D,...,G_previous_1_1.0,G_previous_1_2.0,G_previous_1_3.0,G_previous_1_4.0,G_previous_1_99.0,G_previous_2_1.0,G_previous_2_2.0,G_previous_2_3.0,G_previous_2_4.0,G_previous_2_99.0
0,10000001,0,9,24,24,9.0,2,1,1,3,...,0,0,0,1,0,0,0,0,0,1
1,10000002,0,7,74,74,15.0,2,0,2,3,...,0,1,0,0,0,0,1,0,0,0
2,10000003,0,4,26,26,1.0,1,0,2,1,...,0,1,0,0,0,0,1,0,0,0
3,10000004,0,13,22,22,3.0,2,0,1,1,...,0,0,1,0,0,0,0,0,1,0
4,10000006,0,11,27,27,12.0,0,0,1,1,...,1,0,0,0,0,0,0,1,0,0


In [29]:
print(my_train.shape)
print(my_test.shape)

(97009, 153)
(55716, 153)


## BREAKDOWN OF THIS PROBLEM IN TO A MODEL FOR EACH OPTION NAME

In [30]:
# Model 1 - Predicts G
# Model 2 - Predicts C
# Model 3-  Predicts D ( uses Predicted C as an added column in the model) DECIDED BASED ON CORRELATION
# Model 4-  Predicts A 
# Model 5-  Predicts F ( uses Predicted A as an added column in the model) DECIDED BASED ON CORRELATION
# Model 6-  Predicts B 
# Model 7-  Predicts E ( uses Predicted B as an added column in the model) DECIDED BASED ON CORRELATION

## 2. MODEL TRAINING AND PREDICTION - FUNCTIONS

### REFER TO THE HYPER PARAMETER TUNING RESULTS CODE FOR THESE VALUES

### Model 1 Hyper paprameter tuning results- Prediction of G
Best hyperparameters for XGboost: {'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 72}

Best hyperparameters for random forest: {'max_depth': 9, 'n_estimators': 90}


### Model 2 Hyper paprameter tuning results- Prediction of C

Best hyperparameters for XGboost: {'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 70}

Best hyperparameters for random forest: {'max_depth': 8, 'n_estimators': 154}


### Model 3 Hyper paprameter tuning results- Prediction of D

Best hyperparameters for XGboost: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 107}

Best hyperparameters for random forest: {'max_depth': 9, 'n_estimators': 223}


### Model 4 Hyper paprameter tuning results- Prediction of A

Best hyperparameters for XGboost: {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 87}

Best hyperparameters for random forest: {'max_depth': 7, 'n_estimators': 197}


### Model 5 Hyper paprameter tuning results- Prediction of F

Best hyperparameters for XGboost: {'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 115}

Best hyperparameters for random forest: {'max_depth': 9, 'n_estimators': 247}


### Model 6 Hyper paprameter tuning results- Prediction of B

Best hyperparameters for XGboost: {'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 274}

Best hyperparameters for random forest: {'max_depth': 4, 'n_estimators': 277}


### Model 7 Hyper paprameter tuning results- Prediction of E

Best hyperparameters for XGboost: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 262}

Best hyperparameters for random forest: {'max_depth': 9, 'n_estimators': 256}

In [ ]:
#Below function trains the model
def model_train(data,response_column,cols_to_drop,method='logistic',rf_parameters=None,xgb_parameters=None ):
    
    
    num_class=len(data[response_column].unique())
    y = data[response_column]
    X = data.copy()
    X.drop(cols_to_drop, axis=1, inplace=True)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)
    print("Training data size", X_train.shape)
    print("validation data size", X_val.shape)
    
    if(method=='logistic'):
        print("\nUsing Logistic regression:\n")
        logreg = LogisticRegression(solver='lbfgs', max_iter=300)
        logreg.fit(X_train, y_train)
        y_pred_val=logreg.predict(X_val)
        y_pred_prob_val=logreg.predict_proba(X_val) 
        metrics.confusion_matrix(y_val,y_pred_val)
        print("Classification Report\n",classification_report(y_val,y_pred_val))
        print("Accuracy:  ", accuracy_score(y_val, y_pred_val))
        
    elif(method=='random_forest'):
        print("\nUsing random_forest:\n'")
        rf = RandomForestClassifier(**rf_parameters)
        rf.fit(X_train, y_train)

        y_pred_val=rf.predict(X_val)
        y_pred_prob_val=rf.predict_proba(X_val) 
        metrics.confusion_matrix(y_val,y_pred_val)
        print("Using Random Forest - classification_report",classification_report(y_val,y_pred_val))
        print("Using Random Forest - Overall accuracy", accuracy_score(y_val, y_pred_val))
        
        
    elif(method=='XGBOOST'):
        print("\nUsing XGBOOST:\n'")
        
        #iF TIME PERMITS DO PARAMETER TUNING AND REFINE THESE VALUES - Experiment with different parameters
        xgb_model = xgb.XGBClassifier(**xgb_parameters)
        xgb_model.fit(X_train, y_train)

        y_pred_val=xgb_model.predict(X_val)
        metrics.confusion_matrix(y_val,y_pred_val)
        print("Using XGBOOST - classification_report",classification_report(y_val,y_pred_val))
        print("Using XGBOOST - Overall accuracy", accuracy_score(y_val, y_pred_val))

        return None
    
#Below function predicts on the test data provided
def model_predict(build_data,test_data,response_column,train_cols_to_drop,test_cols_to_drop,method='logistic',rf_parameters=None,xgb_parameters=None):
    
    y = build_data[response_column]
    X = build_data.copy()
    X.drop(train_cols_to_drop, axis=1, inplace=True)
    test_data_new=test_data.drop(test_cols_to_drop, axis=1)
    
    num_class=len(build_data[response_column].unique())
    if(method=='logistic'):
        logreg = LogisticRegression(solver='lbfgs', max_iter=300)
        logreg.fit(X, y)
        output=logreg.predict(test_data_new)

        
    elif(method=='random_forest'):
        rf = RandomForestClassifier(**rf_parameters)
        rf.fit(X, y)
        test_data_new = test_data_new[X.columns]
        output=rf.predict(test_data_new)
    
    
    #Experiment with different parameters
    elif(method=='XGBOOST'):
        xgb_model = xgb.XGBClassifier(**xgb_parameters)
        xgb_model.fit(X, y)
        test_data_new = test_data_new[X.columns]
        output=xgb_model.predict(test_data_new)
        
        
    return output

In [40]:
def combine_sub_models_train(build_data,test_data,method='logistic'):
    
    
    train_cols_to_drop_model_G=['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
    'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_C=['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_D=['record_type','customer_ID','A_purchased','B_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_A=['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_F=['record_type','customer_ID','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_B=['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_E=['record_type','customer_ID','A_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    
    
    
    XGB_params_G= {'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 72}
    rf_params_G={'max_depth': 8, 'n_estimators': 154}

    XGB_params_C= {'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 70}
    rf_params_C={'max_depth': 9, 'n_estimators': 90}

    XGB_params_D= {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 107}
    rf_params_D={'max_depth': 9, 'n_estimators': 223}

    XGB_params_A= {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 87}
    rf_params_A={'max_depth': 7, 'n_estimators': 197}

    XGB_params_F= {'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 115}
    rf_params_F={'max_depth': 9, 'n_estimators': 247}

    XGB_params_B= {'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 274}
    rf_params_B= {'max_depth': 4, 'n_estimators': 277}

    XGB_params_E=  {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 262}
    rf_params_E= {'max_depth': 9, 'n_estimators': 256}

   
    
    print("\nModel 1 Training results- Prediction of G\n\n")
    model_train(build_data,'G_purchased', train_cols_to_drop_model_G,method,rf_params_G,XGB_params_G)
    
    print("\nModel 2 Training results- Prediction of C\n\n")
    model_train(build_data,'C_purchased', train_cols_to_drop_model_C,method,rf_params_C,XGB_params_C)
    
    print("\nModel 3 Training results- Prediction of D\n\n")
    model_train(build_data,'D_purchased', train_cols_to_drop_model_D,method,rf_params_D,XGB_params_D)
    
    print("\nModel 4 Training results- Prediction of A\n\n")
    model_train(build_data,'A_purchased', train_cols_to_drop_model_A,method,rf_params_A,XGB_params_A)
    
    print("\nModel 5 Training results- Prediction of F\n\n")
    model_train(build_data,'F_purchased', train_cols_to_drop_model_F,method,rf_params_F,XGB_params_F)
    
    print("\nModel 6 Training results- Prediction of B\n\n")
    model_train(build_data,'B_purchased', train_cols_to_drop_model_B,method,rf_params_B,XGB_params_B)
    
    print("\nModel 7 Training results- Prediction of E\n\n")
    model_train(build_data,'E_purchased', train_cols_to_drop_model_E,method,rf_params_E,XGB_params_E)
    
    return None

In [54]:
def combine_sub_models_prediction(build_data,test_data,method='logistic'):
    
    train_cols_to_drop_model_G=['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
    'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_C=['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_D=['record_type','customer_ID','A_purchased','B_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_A=['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_F=['record_type','customer_ID','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_B=['record_type','customer_ID','A_purchased','B_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']
    train_cols_to_drop_model_E=['record_type','customer_ID','A_purchased','C_purchased',
        'D_purchased','E_purchased','F_purchased','G_purchased']

    test_cols_to_drop_model_G=['A','B','C','D','E','F','G','record_type','customer_ID']
    test_cols_to_drop_model_C=['A','B','C','D','E','F','G','record_type','customer_ID']
    test_cols_to_drop_model_D=['A','B','C','D','E','F','G','record_type','customer_ID']
    test_cols_to_drop_model_A=['A','B','C','D','E','F','G','record_type','customer_ID']
    test_cols_to_drop_model_F=['A','B','C','D','E','F','G','record_type','customer_ID']
    test_cols_to_drop_model_B=['A','B','C','D','E','F','G','record_type','customer_ID']
    test_cols_to_drop_model_E=['A','B','C','D','E','F','G','record_type','customer_ID']
    
    XGB_params_G= {'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 72}
    rf_params_G={'max_depth': 8, 'n_estimators': 154}

    XGB_params_C= {'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 70}
    rf_params_C={'max_depth': 9, 'n_estimators': 90}

    XGB_params_D= {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 107}
    rf_params_D={'max_depth': 9, 'n_estimators': 223}

    XGB_params_A= {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 87}
    rf_params_A={'max_depth': 7, 'n_estimators': 197}

    XGB_params_F= {'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 115}
    rf_params_F={'max_depth': 9, 'n_estimators': 247}

    XGB_params_B= {'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 274}
    rf_params_B= {'max_depth': 4, 'n_estimators': 277}

    XGB_params_E=  {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 262}
    rf_params_E= {'max_depth': 9, 'n_estimators': 256}



	
    
    output_df=pd.DataFrame(my_test['customer_ID'])
    
    output_df['G_predicted']=model_predict(my_train,my_test,'G_purchased', train_cols_to_drop_model_G,test_cols_to_drop_model_G,
                                           method,rf_params_G,XGB_params_G)
    
    output_df['C_predicted']=model_predict(my_train,my_test,'C_purchased', train_cols_to_drop_model_C,test_cols_to_drop_model_C,
                                           method,rf_params_C,XGB_params_C)
    
    my_test['C_purchased']=output_df['C_predicted']
    output_df['D_predicted']=model_predict(my_train,my_test,'D_purchased', train_cols_to_drop_model_D,test_cols_to_drop_model_D,
                                           method,rf_params_D,XGB_params_D)
    my_test.drop('C_purchased',axis=1,inplace=True)

    output_df['A_predicted']=model_predict(my_train,my_test,'A_purchased', train_cols_to_drop_model_A,test_cols_to_drop_model_A,
                                           method,rf_params_A,XGB_params_A)

    my_test['A_purchased']=output_df['A_predicted']
    output_df['F_predicted']=model_predict(my_train,my_test,'F_purchased', train_cols_to_drop_model_F,test_cols_to_drop_model_F,
                                           method,rf_params_F,XGB_params_F)
    my_test.drop('A_purchased',axis=1,inplace=True)

    output_df['B_predicted']=model_predict(my_train,my_test,'B_purchased', train_cols_to_drop_model_B,test_cols_to_drop_model_B,
                                           method,rf_params_B,XGB_params_B)
      
    my_test['B_purchased']=output_df['B_predicted']
    output_df['E_predicted']=model_predict(my_train,my_test,'E_purchased', train_cols_to_drop_model_E,test_cols_to_drop_model_E,
                                           method,rf_params_E,XGB_params_E)
    my_test.drop('B_purchased',axis=1,inplace=True)

    for C in output_df.columns:
        output_df[C]=output_df[C].astype(str)
    output_df['plan']=output_df['A_predicted'] + output_df['B_predicted'] + output_df['C_predicted'] + output_df['D_predicted'] + output_df['E_predicted'] + output_df['F_predicted'] + output_df['G_predicted']

    return output_df

## 3. MODEL PREDICTIONS - USING LOGISTIC REGRESSION FOR ALL SUB MODELS

### Models training results - using Logistic

In [61]:
combine_sub_models_train(my_train,my_test,'logistic')


Model 1 Training results- Prediction of G


Training data size (77607, 144)
validation data size (19402, 144)

Using Logistic regression:



C:\Users\varsh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report
               precision    recall  f1-score   support

           1       0.81      0.83      0.82      4051
           2       0.84      0.89      0.87      7556
           3       0.91      0.87      0.89      6140
           4       0.85      0.69      0.76      1655

    accuracy                           0.85     19402
   macro avg       0.85      0.82      0.83     19402
weighted avg       0.86      0.85      0.85     19402

Accuracy:   0.8543449128955778

Model 2 Training results- Prediction of C


Training data size (77607, 144)
validation data size (19402, 144)

Using Logistic regression:



C:\Users\varsh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report
               precision    recall  f1-score   support

           1       0.94      0.94      0.94      5799
           2       0.90      0.89      0.90      4078
           3       0.93      0.95      0.94      7593
           4       0.94      0.88      0.91      1932

    accuracy                           0.93     19402
   macro avg       0.93      0.91      0.92     19402
weighted avg       0.93      0.93      0.93     19402

Accuracy:   0.9256262241006081

Model 3 Training results- Prediction of D


Training data size (77607, 145)
validation data size (19402, 145)

Using Logistic regression:



...

C:\Users\varsh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report
               precision    recall  f1-score   support

           0       0.96      0.97      0.96      6249
           1       0.92      0.91      0.92      4793
           2       0.91      0.93      0.92      7234
           3       0.85      0.74      0.79      1126

    accuracy                           0.93     19402
   macro avg       0.91      0.89      0.90     19402
weighted avg       0.93      0.93      0.93     19402

Accuracy:   0.9263477991959592

Model 6 Training results- Prediction of B


Training data size (77607, 144)
validation data size (19402, 144)

Using Logistic regression:



C:\Users\varsh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report
               precision    recall  f1-score   support

           0       0.93      0.95      0.94     10165
           1       0.94      0.92      0.93      9237

    accuracy                           0.93     19402
   macro avg       0.93      0.93      0.93     19402
weighted avg       0.93      0.93      0.93     19402

Accuracy:   0.933975878775384

Model 7 Training results- Prediction of E


Training data size (77607, 145)
validation data size (19402, 145)

Using Logistic regression:

Classification Report
               precision    recall  f1-score   support

           0       0.95      0.95      0.95     10427
           1       0.94      0.94      0.94      8975

    accuracy                           0.94     19402
   macro avg       0.94      0.94      0.94     19402
weighted avg       0.94      0.94      0.94     19402

Accuracy:   0.9445417998144521


C:\Users\varsh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Aggregating Model predictions in to a plan

In [62]:
#Getting the predictions and preparing the kaggle submission file
output_df=combine_sub_models_prediction(my_train,my_test,method='logistic')
final_predictions_df=output_df[['customer_ID','plan']]
final_predictions_df.head()

C:\Users\varsh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\varsh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

,customer_ID,plan
0,10000001,0013131
1,10000002,2023132
2,10000003,1013132
3,10000004,2013133
4,10000006,0013131


In [22]:
#EXPORTING THE FINAL RESULTS AS PDF FOR KAGGLE SUBMISSION
final_predictions_df.to_csv(output_path+ 'Final_logistic_predictions.csv', index=False)

## 4. MODEL PREDICTIONS - USING RANDOM FOREST FOR ALL SUB MODELS

### Models training results - using Random forests

In [43]:
combine_sub_models_train(my_train,my_test,'random_forest')


Model 1 Training results- Prediction of G


Training data size (77607, 144)
validation data size (19402, 144)

Using random_forest:
'
Using Random Forest - classification_report               precision    recall  f1-score   support

           1       0.81      0.85      0.83      4051
           2       0.86      0.89      0.87      7556
           3       0.92      0.87      0.90      6140
           4       0.83      0.74      0.78      1655

    accuracy                           0.86     19402
   macro avg       0.85      0.84      0.85     19402
weighted avg       0.86      0.86      0.86     19402

Using Random Forest - Overall accuracy 0.8628491908050716

Model 2 Training results- Prediction of C


Training data size (77607, 144)
validation data size (19402, 144)

Using random_forest:
'
Using Random Forest - classification_report               precision    recall  f1-score   support

           1       0.94      0.94      0.94      5799
           2       0.90      0.90      0

### Aggregating Model predictions in to a plan

In [55]:
#Getting the predictions and preparing the kaggle submission file


output_df=combine_sub_models_prediction(my_train,my_test,method='random_forest')
final_predictions_df=output_df[['customer_ID','plan']]
final_predictions_df.head()

,customer_ID,plan
0,10000001,1011024
1,10000002,2113122
2,10000003,1013022
3,10000004,2011024
4,10000006,2011021


In [56]:
#EXPORTING THE FINAL RESULTS AS PDF FOR KAGGLE SUBMISSION
final_predictions_df.to_csv(output_path+ 'Final_random_forest_predictions.csv', index=False)

## 5. MODEL PREDICTIONS - USING XGBOOST FOR ALL SUB MODELS

### Models training results - using xgBOOST

In [51]:
combine_sub_models_train(my_train,my_test,'XGBOOST')


Model 1 Training results- Prediction of G


Training data size (77607, 144)
validation data size (19402, 144)

Using XGBOOST:
'
Using XGBOOST - classification_report               precision    recall  f1-score   support

           1       0.81      0.85      0.83      4051
           2       0.86      0.89      0.87      7556
           3       0.92      0.88      0.90      6140
           4       0.83      0.75      0.79      1655

    accuracy                           0.86     19402
   macro avg       0.85      0.84      0.85     19402
weighted avg       0.87      0.86      0.86     19402

Using XGBOOST - Overall accuracy 0.8648592928564065

Model 2 Training results- Prediction of C


Training data size (77607, 144)
validation data size (19402, 144)

Using XGBOOST:
'
Using XGBOOST - classification_report               precision    recall  f1-score   support

           1       0.94      0.94      0.94      5799
           2       0.90      0.90      0.90      4078
           3    

### Aggregating Model predictions in to a plan

In [57]:
#Getting the predictions and preparing the kaggle submission file
output_df=combine_sub_models_prediction(my_train,my_test,method='XGBOOST')
final_predictions_df=output_df[['customer_ID','plan']]
final_predictions_df.head()

,customer_ID,plan
0,10000001,0011004
1,10000002,2023122
2,10000003,1021022
3,10000004,2011023
4,10000006,0011001


In [58]:
#EXPORTING THE FINAL RESULTS AS PDF FOR KAGGLE SUBMISSION
final_predictions_df.to_csv(output_path+ 'Final_xgb_predictions.csv', index=False)